In [1]:
# Continue from the previous script, assuming 'document_store' is populated.
from scripts.indexing import document_store  # Adjust the import as necessary

# Import necessary components for the query pipeline
from haystack.components.embedders import SentenceTransformersTextEmbedder
from haystack.components.retrievers.in_memory import InMemoryEmbeddingRetriever
from haystack.components.builders import PromptBuilder
from haystack.components.generators import OpenAIGenerator
from haystack.utils import Secret
from haystack import Pipeline

/Users/laurafunderburk/Documents/GitHub/Building-Natural-Language-Pipelines/ch4/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Running unified indexing pipeline for web, local files, and CSV...


Error processing document 1384ec36dd6d99f90ab589732d5219b7371dac846d0f0bd89c6385189c4079c0. Keeping it, but skipping cleaning. Error: Error tokenizing data. C error: Expected 5 fields in line 5, saw 7

Error processing document 1384ec36dd6d99f90ab589732d5219b7371dac846d0f0bd89c6385189c4079c0. Keeping it, but skipping splitting. Error: Error tokenizing data. C error: Expected 5 fields in line 5, saw 7

Batches: 100%|██████████| 5/5 [00:03<00:00,  1.58it/s]


In [2]:
# Naive RAG Pipeline Construction

# Text Embedder: To embed the user's query. Must be compatible with the document embedder.
text_embedder = SentenceTransformersTextEmbedder(model="sentence-transformers/all-MiniLM-L6-v2")

# Retriever: Fetches documents from the DocumentStore based on vector similarity.
retriever = InMemoryEmbeddingRetriever(document_store=document_store, top_k=3)

# PromptBuilder: Creates a prompt using the retrieved documents and the query.
# The Jinja2 template iterates through the documents and adds their content to the prompt.
prompt_template_for_pipeline = """
Given the following information, answer the user's question.
If the information is not available in the provided documents, say that you don't have enough information to answer.

Context:
{% for doc in documents %}
    {{ doc.content }}
{% endfor %}

Question: {{question}}
Answer:
"""
prompt_builder_inst = PromptBuilder(template=prompt_template_for_pipeline,
                                    required_variables="*")
llm_generator_inst = OpenAIGenerator(api_key=Secret.from_env_var("OPENAI_API_KEY"), model="gpt-4o-mini")


# --- 2. Build the Naive RAG Pipeline ---

naive_rag_pipeline = Pipeline()

# Add components to the pipeline
naive_rag_pipeline.add_component("text_embedder", text_embedder)
naive_rag_pipeline.add_component("retriever", retriever)
naive_rag_pipeline.add_component("prompt_builder", prompt_builder_inst)
naive_rag_pipeline.add_component("llm", llm_generator_inst)

# --- 3. Connect the Components ---

# The query embedding is sent to the retriever
naive_rag_pipeline.connect("text_embedder.embedding", "retriever.query_embedding")
# The retriever's documents are sent to the prompt builder
naive_rag_pipeline.connect("retriever.documents", "prompt_builder.documents")
# The final prompt is sent to the LLM
naive_rag_pipeline.connect("prompt_builder.prompt", "llm.prompt")


🚅 Components
  - text_embedder: SentenceTransformersTextEmbedder
  - retriever: InMemoryEmbeddingRetriever
  - prompt_builder: PromptBuilder
  - llm: OpenAIGenerator
🛤️ Connections
  - text_embedder.embedding -> retriever.query_embedding (list[float])
  - retriever.documents -> prompt_builder.documents (list[Document])
  - prompt_builder.prompt -> llm.prompt (str)

In [3]:
from haystack.components.retrievers.in_memory import InMemoryBM25Retriever
from haystack.components.joiners import DocumentJoiner
from haystack.components.rankers import TransformersSimilarityRanker

In [4]:
# Hybrid RAG Pipeline Construction

# Text Embedder: To embed the user's query. Must be compatible with the document embedder.
text_embedder = SentenceTransformersTextEmbedder(model="sentence-transformers/all-MiniLM-L6-v2")

# Retriever: Fetches documents from the DocumentStore based on vector similarity.
retriever = InMemoryEmbeddingRetriever(document_store=document_store, top_k=3)

# PromptBuilder: Creates a prompt using the retrieved documents and the query.
# The Jinja2 template iterates through the documents and adds their content to the prompt.
prompt_template_for_pipeline = """
Given the following information, answer the user's question.
If the information is not available in the provided documents, say that you don't have enough information to answer.

Context:
{% for doc in documents %}
    {{ doc.content }}
{% endfor %}

Question: {{question}}
Answer:
"""
prompt_builder_inst = PromptBuilder(template=prompt_template_for_pipeline,
                                    required_variables="*")
llm_generator_inst = OpenAIGenerator(api_key=Secret.from_env_var("OPENAI_API_KEY"), model="gpt-4o-mini")



# Sparse Retriever (BM25): For keyword-based search.
# This retriever needs to be "warmed up" by calculating statistics on the documents in the store.
bm25_retriever = InMemoryBM25Retriever(document_store=document_store, top_k=3)

# DocumentJoiner: To merge the results from the two retrievers.
# The default 'concatenate' mode works well here as the ranker will handle final ordering.
document_joiner = DocumentJoiner()

# Ranker: A cross-encoder model to re-rank the combined results for higher precision.
# This model is highly effective at identifying the most relevant documents from a candidate set.
ranker = TransformersSimilarityRanker(model="BAAI/bge-reranker-base", top_k=3)


# --- 2. Build the Hybrid RAG Pipeline ---

hybrid_rag_pipeline = Pipeline()

# Add all necessary components
hybrid_rag_pipeline.add_component("text_embedder", text_embedder)
hybrid_rag_pipeline.add_component("embedding_retriever", retriever) # Dense retriever
hybrid_rag_pipeline.add_component("bm25_retriever", bm25_retriever) # Sparse retriever
hybrid_rag_pipeline.add_component("document_joiner", document_joiner)
hybrid_rag_pipeline.add_component("ranker", ranker)
hybrid_rag_pipeline.add_component("prompt_builder", prompt_builder_inst)
hybrid_rag_pipeline.add_component("llm", llm_generator_inst)

# --- 3. Connect the Components in a Graph ---

# The query is embedded for the dense retriever
hybrid_rag_pipeline.connect("text_embedder.embedding", "embedding_retriever.query_embedding")

# The raw query text is sent to the BM25 retriever and the ranker
# Note: The query input for these components is the raw text string.

# The outputs of both retrievers are fed into the document joiner
hybrid_rag_pipeline.connect("embedding_retriever.documents", "document_joiner.documents")
hybrid_rag_pipeline.connect("bm25_retriever.documents", "document_joiner.documents")

# The joined documents are sent to the ranker
hybrid_rag_pipeline.connect("document_joiner.documents", "ranker.documents")

# The ranked documents are sent to the prompt builder
hybrid_rag_pipeline.connect("ranker.documents", "prompt_builder.documents")

# The final prompt is sent to the LLM
hybrid_rag_pipeline.connect("prompt_builder.prompt", "llm.prompt")


TransformersSimilarityRanker is considered legacy and will no longer receive updates. It may be deprecated in a future release, with removal following after a deprecation period. Consider using SentenceTransformersSimilarityRanker instead, which provides the same functionality along with additional features.


🚅 Components
  - text_embedder: SentenceTransformersTextEmbedder
  - embedding_retriever: InMemoryEmbeddingRetriever
  - bm25_retriever: InMemoryBM25Retriever
  - document_joiner: DocumentJoiner
  - ranker: TransformersSimilarityRanker
  - prompt_builder: PromptBuilder
  - llm: OpenAIGenerator
🛤️ Connections
  - text_embedder.embedding -> embedding_retriever.query_embedding (list[float])
  - embedding_retriever.documents -> document_joiner.documents (list[Document])
  - bm25_retriever.documents -> document_joiner.documents (list[Document])
  - document_joiner.documents -> ranker.documents (list[Document])
  - ranker.documents -> prompt_builder.documents (list[Document])
  - prompt_builder.prompt -> llm.prompt (str)

In [7]:
import pandas as pd 
# Convert the generated test set to a pandas DataFrame for easier handling
test_df = pd.read_csv("./data_for_eval/synthetic_qa_pairs.csv")
print("Generated Test Set:")
test_df.head()

Generated Test Set:


,user_input,reference_contexts,reference,synthesizer_name
0,What is OpenAI?,['Introduction\nChatGPT launched in November 2...,"OpenAI is the developer of ChatGPT, a large la...",single_hop_specific_query_synthesizer
1,How many total messages were sent in Jun 2025?,['Month\nNon-Work (M)\n(%)\nWork (M)\n(%)\nTot...,"The total messages sent in Jun 2025 were 2,627.",single_hop_specific_query_synthesizer
2,What is the significance of June 2025 in ChatG...,['Table 1: ChatGPT daily message counts (milli...,The context reports data ending on the 26th of...,single_hop_specific_query_synthesizer
3,How does Caplin relate to AI's economic value?,"['Doing, and that Asking messages are consiste...",The context discusses how ChatGPT improves wor...,single_hop_specific_query_synthesizer
4,How do inter-annotator agreement metrics like ...,"[""<1-hop>\n\nAppendix: Classifier Validation\n...",The context shows that inter-annotator agreeme...,multi_hop_abstract_query_synthesizer


In [ ]:
hybrid_rag_pipeline.

In [ ]:
def get_pipeline_outputs(pipeline, questions):
    """Helper function to run a pipeline over a list of questions and collect outputs."""
    outputs = []
    for q in questions:
        if "bm25_retriever" in pipeline.: # Hybrid pipeline
            result = pipeline.run({
                "text_embedder": {"text": q},
                "bm25_retriever": {"query": q},
                "ranker": {"query": q},
                "prompt_builder": {"question": q}
            })
            # For hybrid, the final documents come from the ranker
            retrieved_docs = result["ranker"]["documents"]
        else: # Naive pipeline
            result = pipeline.run({
                "text_embedder": {"text": q},
                "prompt_builder": {"question": q}
            })
            retrieved_docs = result["retriever"]["documents"]
        
        answer = result["llm"]["replies"].content
        contexts = [doc.content for doc in retrieved_docs]
        outputs.append({"answer": answer, "contexts": contexts})
    return outputs

questions = test_df["user_input"].tolist()
ground_truths = test_df["reference"].tolist()

print("\nRunning Naive RAG pipeline...")
naive_outputs = get_pipeline_outputs(naive_rag_pipeline, questions)

print("Running Hybrid RAG pipeline...")
hybrid_outputs = get_pipeline_outputs(hybrid_rag_pipeline, questions)




Running Naive RAG pipeline...


AttributeError: 'Pipeline' object has no attribute 'components'

In [ ]:
# --- 3. Build the Evaluation Pipeline ---

from haystack_integrations.components.evaluators.ragas import RagasEvaluator, RagasMetric
# Initialize RagasEvaluator for each metric
faithfulness_evaluator = RagasEvaluator(metric=RagasMetric.FAITHFULNESS)
context_recall_evaluator = RagasEvaluator(metric=RagasMetric.CONTEXT_RECALL)

# Create the evaluation pipeline
evaluation_pipeline = Pipeline()
evaluation_pipeline.add_component("faithfulness", faithfulness_evaluator)
evaluation_pipeline.add_component("context_recall", context_recall_evaluator)

# --- 4. Execute Evaluation and Analyze Results ---

def run_evaluation(pipeline_outputs):
    """Helper function to run the evaluation pipeline and return scores."""
    eval_results = evaluation_pipeline.run({
        "faithfulness": {
            "questions": questions,
            "contexts": [out["contexts"] for out in pipeline_outputs],
            "responses": [out["answer"] for out in pipeline_outputs]
        },
        "context_recall": {
            "questions": questions,
            "contexts": [out["contexts"] for out in pipeline_outputs],
            "ground_truths": ground_truths
        }
    })
    
    # Calculate average scores
    avg_faithfulness = sum(r["score"] for r in eval_results["faithfulness"]["results"]) / len(eval_results["faithfulness"]["results"])
    avg_context_recall = sum(r["score"] for r in eval_results["context_recall"]["results"]) / len(eval_results["context_recall"]["results"])
    
    return {"faithfulness": avg_faithfulness, "context_recall": avg_context_recall}

print("\nEvaluating Naive RAG pipeline...")
naive_scores = run_evaluation(naive_outputs)

print("Evaluating Hybrid RAG pipeline...")
hybrid_scores = run_evaluation(hybrid_outputs)

# --- Display Results in a Table ---
results_df = pd.DataFrame({
    "Metric":,
    "Naive RAG Score": [naive_scores["faithfulness"], naive_scores["context_recall"]],
    "Hybrid RAG Score": [hybrid_scores["faithfulness"], hybrid_scores["context_recall"]]
})

print("\n--- Ragas Evaluation Results ---")
print(results_df.to_string(index=False))